# Import bibliotek

Projekt rozpoczniemy od zaimportowania potrzebnych bibliotek: 
-Tkinter – biblioteka umożliwiająca tworzenie interfejsu graficznego, 
-cv jest używana do wizji komputerowej w sztucznej inteligencji, uczeniu maszynowym, rozpoznawaniu twarzy, 
-numpy dodaje możliwość obsługi dużych, wielowymiarowych tabel i macierzy,
-tensorflow - wykorzystywana jest w uczeniu maszynowym i głębokich sieciach neuronowych, 
-keras - zapewnia interfejs Pythona dla sztucznych sieci neuronowych.

In [1]:
import os
import threading
import tkinter as tk
from tkinter import *

import cv2
import numpy as np
from PIL import Image, ImageTk
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Sequential

# Plik treningowy

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Tworzymy katalogi dla danych treningowych oraz testowych, w nich przechowywać będziemy pliki graficzne przedstawiające wyrazy twarzy. Wykorzystano aż 28709 danych uczących oraz 7178 danych testowych. Tworzymy generator danych obrazu, który wygeneruje zestaw danych z plików graficznych w skali od 1 do 255. Nasze obrazy definiujemy w pikselach, czyli tworzymy tablice, które przechowywują wartości od 1 do 255. Ustawiamy docelowy rozmiar na 48x48 pikseli. Tryb koloru ustawiamy na odcienie szarości, tryb klasowy na kategoryczny, ponieważ będziemy kategoryzować otrzymane dane np. zły, uśmiechnięty. 

In [6]:
train_dir = 'data/train'
test_dir = 'data/test'
train_gen_data = ImageDataGenerator(rescale=1. / 255)
train_test_data = ImageDataGenerator(rescale=1. / 255)
train_generator = train_gen_data.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)
test_generator = train_gen_data.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


Tworzmy model oraz przygotowujemy go do treningu. Zaczynamy od zainicjiwania modelu sekwencyjnego z wykorzystaniem wcześniej zaimportowanej biblioteki keras, takie rozwiązanie umożliwoa tworzenie wartswy po warstwie. Za pomocą metody add modelujemy naszą sieć neuronować dodając poszczególne parametry.

In [8]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

Teraz przyszedł czas na kompilację oraz zapisanie naszego modelu. Na samym kończu zostaną zapisane wagi do pliku model.h5.

In [9]:
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])
emotion_model.info = emotion_model.fit_generator(
    train_generator,
    steps_per_epoch=28709 // 64,
    epochs=50,
    validation_data=test_generator,
    validation_steps=7178 // 64
)
emotion_model.save_weights('model.h5')

C:\Users\Mateusz\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
C:\Users\Mateusz\AppData\Local\Temp\ipykernel_10556\1724833173.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model.info = emotion_model.fit_generator(


Epoch 1/50
448/448 [==============================] - 251s 559ms/step - loss: 1.7996 - accuracy: 0.2622 - val_loss: 1.7165 - val_accuracy: 0.3504
Epoch 2/50
448/448 [==============================] - 150s 334ms/step - loss: 1.6311 - accuracy: 0.3665 - val_loss: 1.5472 - val_accuracy: 0.4097
Epoch 3/50
448/448 [==============================] - 168s 374ms/step - loss: 1.5317 - accuracy: 0.4120 - val_loss: 1.4666 - val_accuracy: 0.4407
Epoch 4/50
448/448 [==============================] - 170s 381ms/step - loss: 1.4663 - accuracy: 0.4401 - val_loss: 1.4146 - val_accuracy: 0.4671
Epoch 5/50
448/448 [==============================] - 165s 369ms/step - loss: 1.4129 - accuracy: 0.4584 - val_loss: 1.3739 - val_accuracy: 0.4766
Epoch 6/50
448/448 [==============================] - 168s 376ms/step - loss: 1.3590 - accuracy: 0.4820 - val_loss: 1.3346 - val_accuracy: 0.4976
Epoch 7/50
448/448 [==============================] - 180s 401ms/step - loss: 1.3175 - accuracy: 0.5024 - val_loss: 1.3053 -

# Plik emocji

Do pliku emocji dołączamy model sekwencyjny...

Tworzymy słownik z wszystkich emocji, które planujemy rozpoznawać. Importujemy obrazki, które będą prezentować rozpoznany humor.  Tworzymy zmienne globalne, które będziemy wykorzystywać do czytania klatek po kolei.

In [12]:
cv2.ocl.setUseOpenCL(False)

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

cur_path = os.path.abspath(os.curdir)
emoji_dist = {0: cur_path + "/emojis/angry.png", 1: cur_path + "/emojis/disgusted.png",
              2: cur_path + "/emojis/fearful.png", 3: cur_path + "/emojis/happy.png",
              4: cur_path + "/emojis/neutral.png", 5: cur_path + "/emojis/sad.png",
              6: cur_path + "/emojis/surprised.png"}

global last_frame1
last_frame1 = np.zeros((480, 640, 3), dtype=np.uint8)
global cap1
show_text = [0]
global frame_number

W pierwszym kroku otwieramy nasze wideo. Następnie obliczamy długość naszych ramek. Ważnym elementem metody jest zmiana ramki z 600 na 500, dzięki temu rozwiązaniu mamy pewność, że ramka została odczytana. Zmienna bounding_box to ramka, która znajduje się obok twarzy osoby z filmiku. Potem konwertujemy obraz z filmiku na odcienie szarości, aby wykorzystać wcześniej przygotowany model klasyfikacji, taka konwersja pozwala nam zaoszczędzić dużo miejsca w pamięci w porównaniu do kolorowych zdjęć. Na samym końcu naszej metody aktualizujemy okno główne, aby uzytkać najnowsze dane. Ważnym krokiem tej metody jest konwersja obrazu na RGB. Na końcu opóźniamy wywyłanie funcji o 10 milisekund.

In [13]:
def show_subject():
    cap1 = cv2.VideoCapture(r'example2.mp4')
    if not cap1.isOpened():
        print("Can't open the camera")
    global frame_number
    length = int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_number += 1
    if frame_number >= length:
        exit()
    cap1.set(1, frame_number)
    flag1, frame1 = cap1.read()
    frame1 = cv2.resize(frame1, (600, 500))
    bounding_box = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame1, (x, y - 50), (x + w, y + h + 10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame1, emotion_dict[maxindex], (x + 20, y - 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
        show_text[0] = maxindex
    if flag1 is None:
        print("Major error!")
    elif flag1:
        global last_frame1
        last_frame1 = frame1.copy()
        pic = cv2.cvtColor(last_frame1, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(pic)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        root.update()
        lmain.after(10, show_subject)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        exit()

W tej metodzie odczytujemy wartość emocji, którą chcemy przekonwertować na RGB. Otwwieramy odpowiadający obrazek oraz ładujemy go do etykiety interfejsu. Aktualizujemy katalog główny oraz wywołujemy tą samą funkcję opóźnioną o 10 milisekund.

In [14]:
def show_avatar():
    frame2 = cv2.imread(emoji_dist[show_text[0]])
    pic2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
    img2 = Image.fromarray(frame2)
    imgtk2 = ImageTk.PhotoImage(image=img2)
    lmain2.imgtk2 = imgtk2
    lmain3.configure(text=emotion_dict[show_text[0]], font=('arial', 45, 'bold'))
    lmain2.configure(image=imgtk2)
    root.update()
    lmain2.after(10, show_avatar)

Inicjujemy ramkę główną oraz potrzebne labele, które będą zawierały załadowe zdjęcia lub film oraz ustawiemy parametru interfejsu programu. 

In [16]:
if __name__ == '__main__':
    frame_number = 0
    root = tk.Tk()
    lmain = tk.Label(master=root, padx=50, bd=10)
    lmain2 = tk.Label(master=root, bd=10)
    lmain3 = tk.Label(master=root, bd=10, fg="#CDCDCD", bg="black")
    lmain.pack(side=LEFT)
    lmain.place(x=50, y=250)
    lmain3.pack()
    lmain3.place(x=960, y=250)
    lmain2.pack()
    lmain2.place(x=960, y=350)

    root.title("Photo to Emoji")
    root.geometry("1400x900+100+10")
    root['bg'] = 'black'
    Button(root, text="Quit", fg="red", command=root.destroy, font=("arial", 25, "bold")).pack(side=BOTTOM)
    threading.Thread(target=show_subject).start()
    threading.Thread(target=show_avatar).start()
    root.mainloop()

1/1 [==============================] - 0s 24ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Mateusz\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Mateusz\anaconda3\lib\tkinter\__init__.py", line 814, in callit
    func(*args)
  File "C:\Users\Mateusz\AppData\Local\Temp\ipykernel_10556\2591147030.py", line 32, in show_subject
    imgtk = ImageTk.PhotoImage(image=img)
  File "C:\Users\Mateusz\anaconda3\lib\site-packages\PIL\ImageTk.py", line 112, in __init__
    self.__photo = tkinter.PhotoImage(**kw)
  File "C:\Users\Mateusz\anaconda3\lib\tkinter\__init__.py", line 4064, in __init__
    Image.__init__(self, 'photo', name, cnf, master, **kw)
  File "C:\Users\Mateusz\anaconda3\lib\tkinter\__init__.py", line 3997, in __init__
    master = _get_default_root('create image')
  File "C:\Users\Mateusz\anaconda3\lib\tkinter\__init__.py", line 297, in _get_default_root
    raise RuntimeError(f"Too early to {what}: no default root window")
RuntimeErr